In [5]:
from skimage import data
import time
import numpy as np
import napari
from PIL import Image, ImageOps
from napari.qt import thread_worker
from qtpy.QtCore import Qt
from qtpy.QtWidgets import QPushButton

In [6]:
tsr = Image.open("/Users/homelyp/Desktop/ghos.tif") 

In [7]:
width, height = tsr.size #X and Y image dimensions
#chose desired image width, for example, a width of 200 should be used if you have a slow machine
X_dim = width #chose "width" if you want to use the whole image
Y_dim = int(height*(X_dim/width)) #maintain aspect ration as original image

#get the data into a numpy array
images = []
for i in range(tsr.n_frames):
    tsr.seek(i)
    images.append(np.array(ImageOps.fit(tsr, (X_dim,Y_dim)),dtype=np.float32 )/ 255 ) #ImageOps changes image size 

In [149]:
#shape of array after capture
# scan number is dimension 0
# wavelength dimension is dimension 1
# spatial dimension in dimension 2


np.shape(images)

(493, 240, 676)

In [151]:
#Transpose dimensions of a numpy array
#such that wavelength is dimension 0,
#scan number is dimension 1, and
#spatial dimension is 2

wave_first = np.transpose(images, (1, 0, 2)) 
np.shape(wave_first)

(240, 493, 676)

In [193]:
np.shape(wave_first[1])

(493, 676)

In [80]:
#how to add numpy array to Napari
v.add_image(arr_reshaped[22], name='slice22')

<Image layer 'slice22' at 0x7fdc78b0e580>

In [183]:
#croping and averaging for white balance

#step 1 use "Shapes" in Napari to draw a bounding box around white balance target


def Mean_White_Balance(image):
    
    box = v.layers['Shapes'].data 
    
    #dim1 = np.shape(image)[0]
    dim2 = np.shape(image[0][int(box[0][0][0]):int(box[0][2][0]), int(box[0][0][1]):int(box[0][2][1])])[0]
    dim3 = np.shape(image[0][int(box[0][0][0]):int(box[0][2][0]), int(box[0][0][1]):int(box[0][2][1])])[1]
    
    wb = []#np.zeros((dim1,dim2,dim3))
    
    for i in range(np.shape(image)[0]):
        wb.append(np.mean(image[i][int(box[0][0][0]):int(box[0][2][0]), int(box[0][0][1]):int(box[0][2][1])]))
    
    return(wb)


#step 2 run "Mean_White_Balance" to return mean white balance as below:

mean_white_balance = Mean_White_Balance(wave_first)


    

In [204]:
images2 = np.transpose(images, (0, 2, 1))/np.array(mean_white_balance)

In [205]:
np.shape(images2)

(493, 676, 240)

In [197]:
np.shape(np.array(mean_white_balance))

(240,)

In [93]:
v.add_image(cropped, name='slice22 cropped')

<Image layer 'slice22 cropped [1]' at 0x7fdca82b9a90>

In [8]:
scan = np.zeros((493,676,3))#initial blank image for display in Napari.
#Spatial dimensionsis 676 (in this example). This should be a fixed value for the spectral camera. 
#493 is the number of images collected. 3 is the 3-channel RGB dimension

@thread_worker
def collect_scan(layer, interval=0.05):
    
    update_scan = np.zeros((493,676,3)) #numpy array where color data is added for display in Napari 
    
    
    for i in range(492): #for loop takes one through the entire scan range
        
        #images = camera.capture() #capture with Asi ZWO camera
        
        buffer=np.zeros((1,676,3)) #Buffer to hold sudo-color image for single scan line
        
        buffer[:,:,0] = images[i][120] #red
        buffer[:,:,1] = images[i][90] #green
        buffer[:,:,2] = images[i][60] #blue
        
        
        
        update_scan[i] = buffer #add sudo-color scan line to update array
        
        layer.data = update_scan #update layer in Napari so one can visualize scan
        yield
        time.sleep(interval)

    

with napari.gui_qt():
    v = napari.Viewer()#(show=False)
    #v.window._qt_window.setWindowState(Qt.WindowMaximized)
    #v.show()

    image_layer = v.add_image(scan)
    worker = collect_scan(image_layer)
    
    #worker.start()

    button = QPushButton("Start Scan")
    button.clicked.connect(worker.start)
    v.window.add_dock_widget(widget=button, area='right')

/Users/homelyp/opt/anaconda3/lib/python3.8/site-packages/napari/_qt/qt_event_loop.py:284: FutureWarning: 
The 'gui_qt()' context manager is deprecated.
If you are running napari from a script, please use 'napari.run()' as follows:

    import napari

    viewer = napari.Viewer()  # no prior setup needed
    # other code using the viewer...
    napari.run()

In IPython or Jupyter, 'napari.run()' is not necessary. napari will automatically
start an interactive event loop for you: 

    import napari
    viewer = napari.Viewer()  # that's it!

  warn(
(Make sure 'QVector<int>' is registered using qRegisterMetaType().)
/Users/homelyp/opt/anaconda3/lib/python3.8/site-packages/napari/layers/base/base.py:1632: RuntimeWarning: invalid value encountered in cast
  corners[:, displayed_axes] = data_bbox_clipped
/Users/homelyp/opt/anaconda3/lib/python3.8/site-packages/napari/layers/base/base.py:1632: RuntimeWarning: invalid value encountered in cast
  corners[:, displayed_axes] = data_bbox_clipped

In [83]:
kop= v.layers['Shapes'].data

In [84]:
print(kop)

[array([[173.36105126, 194.96621719],
       [367.92728803, 194.96621719],
       [367.92728803, 390.97368535],
       [173.36105126, 390.97368535]])]


In [91]:
int(kop[0][0][0])

367